In [4]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary

!pip install torchsummary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

In [13]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3) #input - 28*28*1, Channels - 32, Output - 28*28*32 Parameters - 1*1*32*32
        self.bn1 = nn.BatchNorm2d(32)
        self.dpt1 = nn.Dropout(0.25)

        self.conv2 = nn.Conv2d(32, 64, 3)  # Input - 28*28*32, Channels - 64, Output - 26*26*64 - Para - 9*32*64 + 64
        self.dpt2 = nn.Dropout(0.25)
        self.bn2 = nn.BatchNorm2d(64)  # Normalization of features - 
        self.pool1 = nn.MaxPool2d(2, 2)  # Maxpool2D

        self.conv3 = nn.Conv2d(64, 32, 3)  # Input - 26*26*64, Channels - 64, Output - 13*13*64 - Para - 9*64*64 + 64
        self.dpt3 = nn.Dropout(0.25)
        self.bn3 = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(32, 64, 3)  # input -28*28*32 OUtput - 32 - Para - 9*32*32
        self.dpt4 = nn.Dropout(0.25)
        self.bn4 = nn.BatchNorm2d(64)  # input -28*28*32 OUtput - 32 - Para - 9*32*32
        self.conv5 = nn.Conv2d(64, 32, 3)  # input -28*28*32 OUtput - 32 - Para - 9*32*32
        self.dpt5 = nn.Dropout(0.25)
        self.bn5 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2, 2)  # input -28*28*32 OUtput - 32 - Para - 9*32*32
        

        self.conv6 = nn.Conv2d(32, 32, 1)  # input -28*28*32 OUtput - 32 - Para - 9*32*32
        self.conv7 = nn.Conv2d(32, 10, 3)  # input -28*28*32 OUtput - 32 - Para - 9*32*32


    def forward(self,x):
        x = self.dpt1(F.relu(self.bn1(self.conv1(x))))
        x = self.dpt2(self.pool1(self.bn2(F.relu(self.conv2(x)))))
        x = self.dpt3(self.bn3(F.relu(self.conv3(x))))
        x = self.dpt4(self.bn4(F.relu(self.conv4(x))))
        x = self.pool2(self.dpt5(self.bn5(F.relu(self.conv5(x)))))
        x = F.relu(self.conv6(x))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [14]:

model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 26, 26]             320
       BatchNorm2d-2           [-1, 32, 26, 26]              64
           Dropout-3           [-1, 32, 26, 26]               0
            Conv2d-4           [-1, 64, 24, 24]          18,496
       BatchNorm2d-5           [-1, 64, 24, 24]             128
         MaxPool2d-6           [-1, 64, 12, 12]               0
           Dropout-7           [-1, 64, 12, 12]               0
            Conv2d-8           [-1, 32, 10, 10]          18,464
       BatchNorm2d-9           [-1, 32, 10, 10]              64
          Dropout-10           [-1, 32, 10, 10]               0
           Conv2d-11             [-1, 64, 8, 8]          18,496
      BatchNorm2d-12             [-1, 64, 8, 8]             128
          Dropout-13             [-1, 64, 8, 8]               0
           Conv2d-14             [-1, 3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [7]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [8]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n' .format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [15]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(19):
    print('EPOCH: ', epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]

EPOCH:  0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:39: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0511, Accuracy: 9857/10000 (99%)

EPOCH:  1


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0179, Accuracy: 9947/10000 (99%)

EPOCH:  17


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0182, Accuracy: 9944/10000 (99%)

EPOCH:  18


loss=0.01016639731824398 batch_id=468: 100%|██████████| 469/469 [00:22<00:00, 20.70it/s]



Test set: Average loss: 0.0172, Accuracy: 9937/10000 (99%)

